In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [21]:
# Constants
NUM_GAMES = 1000
BATCH_SIZE = 32
LR = 1e-3

In [22]:
df = pd.read_csv('champions.csv')
def get_champ_vec(champ_name):
    arr = []
    row = df.loc[df['Champion'] == champ_name].iloc[0]
    for i in range(1, 16):
        if i == 7:
            if row[i] == 1:
                arr.append(1)
                arr.append(0)
                arr.append(0)
            elif row[i] == 2:
                arr.append(0)
                arr.append(1)
                arr.append(0)
            elif row[i] == 3:
                arr.append(0)
                arr.append(0)
                arr.append(1)
        else:
            arr.append(row[i] / 10)
    return arr

In [23]:
def process_match(match_index):
    with open('data/matches/' + str(match_index) + '.txt', 'r') as f:
        rows = f.read().split('\n')
        vec = []
        for j in range(10):
            champ_vec = get_champ_vec(rows[j])
            for e in champ_vec:
                vec.append(e)
        if rows[10] == 'true':
            vec.append(1)
        else:
            vec.append(0)
        return vec

In [46]:
def process_row(x):
    return (tf.cast(x[:-1], 'float32'), tf.cast(x[-1], 'int32'))

In [47]:
def get_dataset():
    tmp = []
    for i in range(NUM_GAMES):
        tmp.append(process_match(i))
    ds = tf.data.Dataset.from_tensor_slices(tmp)
    ds = ds.map(process_row)
    ds = ds.cache()
    ds = ds.shuffle(NUM_GAMES)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [48]:
ds = get_dataset()

In [49]:
# Partition Dataset
train_size = int(len(ds)*0.7)
val_size = int(len(ds)*0.2)
test_size = int(len(ds)*0.1)

train = ds.take(train_size)
val = ds.skip(train_size).take(val_size)
test = ds.skip(train_size+val_size).take(test_size)

In [67]:
def create_model():
    model = keras.Sequential()
    model.add(layers.Dense(170, activation='relu', input_dim=170))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1))
    return model

In [68]:
model = create_model()

In [69]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 170)               29070     
                                                                 
 dense_13 (Dense)            (None, 64)                10944     
                                                                 
 dense_14 (Dense)            (None, 16)                1040      
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 41,071
Trainable params: 41,071
Non-trainable params: 0
_________________________________________________________________


In [70]:
loss = losses.BinaryCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=LR)

In [71]:
model.compile(loss=loss, optimizer=optimizer, metrics=[tf.metrics.BinaryAccuracy(threshold=0.0)])

In [72]:
history = model.fit(train, validation_data=val, epochs=20)

Epoch 1/20
22/22 [==============================] - 1s 28ms/step - loss: nan - binary_accuracy: 0.5185 - val_loss: nan - val_binary_accuracy: 0.4531
Epoch 2/20
22/22 [==============================] - 0s 16ms/step - loss: nan - binary_accuracy: 0.5000 - val_loss: nan - val_binary_accuracy: 0.5052
Epoch 3/20
22/22 [==============================] - 0s 11ms/step - loss: nan - binary_accuracy: 0.5099 - val_loss: nan - val_binary_accuracy: 0.5260
Epoch 4/20
22/22 [==============================] - 0s 11ms/step - loss: nan - binary_accuracy: 0.5000 - val_loss: nan - val_binary_accuracy: 0.4792
Epoch 5/20
22/22 [==============================] - 0s 11ms/step - loss: nan - binary_accuracy: 0.5085 - val_loss: nan - val_binary_accuracy: 0.4740
Epoch 6/20
22/22 [==============================] - 0s 11ms/step - loss: nan - binary_accuracy: 0.5114 - val_loss: nan - val_binary_accuracy: 0.4948
Epoch 7/20
22/22 [==============================] - 0s 11ms/step - loss: nan - binary_accuracy: 0.5057 - v

In [63]:
X = next(iter(train))